In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from operator import truediv

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
test_preds = pd.read_pickle("/mnt/data/Users/A-PROOF/Cecilia_S/models/fine_tuned/binary/test_set_pos_lq.pkl")

In [4]:
test_preds

,index,pad_sen_id,institution,year,MDN,NotitieID,batch,annotator,background_sent,target_sent,text_raw,original_labels,text,labels_9,labels_binary,labels_10,pred_best_model,pred_lq_data1,confidence
821,6810,237672660_0033,amc,2018,6223307,237672660,week_18,katsburg,False,False,"Anamnese M is wel nerveus over dit gesprek , bang dat we iets doorvertellen aan instanties .","[0, 0, 0, 0, 0, 0, 0, 0, 1]","Anamnese M is wel nerveus over dit gesprek , bang dat we iets doorvertellen aan instanties .","[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09358591870924066, 0.09325441701394238, 0.09341207319591131, 0.09336631342654787, 0.0936135816012269, 0.09385391033891609, 0.09356140714683492, 0.09322195570256535, 0.2521304228648145]"
827,6895,237672660_0039,amc,2018,6223307,237672660,week_18,katsburg,False,False,"M is niet bang , wil ook geen aangifte of scheiding .","[0, 0, 0, 0, 0, 0, 0, 0, 1]","M is niet bang , wil ook geen aangifte of scheiding .","[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09358507025914722, 0.09324724125140077, 0.09344117951243254, 0.0933398217249897, 0.09361093802412519, 0.09388729656664592, 0.09355342069501008, 0.09321633326790207, 0.2521186986983465]"
831,6939,237672660_0043,amc,2018,6223307,237672660,week_18,katsburg,False,False,"Heeft het vechten nooit gezien , maar ziet wel dat moeder dan boos is , heeft het mogelijk ook gehoord .","[0, 0, 0, 0, 0, 0, 0, 0, 1]","Heeft het vechten nooit gezien , maar ziet wel dat moeder dan boos is , heeft het mogelijk ook gehoord .","[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09363395734373231, 0.0932535713632954, 0.0934284485580349, 0.09333472148369376, 0.0935956190175647, 0.09388262568354304, 0.09352878612617939, 0.09321331525524418, 0.2521289551687123]"
832,6960,237672660_0044,amc,2018,6223307,237672660,week_18,katsburg,False,False,"Priscilla is nooit geslagen , P is niet boos op de kinderen , alleen op M. M slaat ook niet .","[0, 0, 0, 0, 0, 0, 0, 0, 0]","PERSON is nooit geslagen , P is niet boos op de kinderen , alleen op PERSON slaat ook niet .","[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09361482316815592, 0.09321437383703894, 0.09343594688594525, 0.09331599384787412, 0.09360579156821942, 0.09402368259885356, 0.09352643200503734, 0.09318338190398291, 0.25207957418489246]"
834,6991,237672660_0046,amc,2018,6223307,237672660,week_18,katsburg,False,False,Priscilla lijkt wel bezorgd .,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",PERSON lijkt wel bezorgd .,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09359741542082294, 0.09330687468359604, 0.09344553446433176, 0.09336850003847856, 0.0935984449841024, 0.09377175291195286, 0.09353798381742151, 0.09326806034041701, 0.2521054333388768]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215368,2010031,439612646_0095,amc,2020,3850656,439612646,ze_batch1,hinke,False,False,Ondervoeding - SNAQ Score,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",Ondervoeding - SNAQ Score,"[0, 0, 0, 0, 0, 0, 0, 1, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","[0.09438871878799326, 0.09400581266213694, 0.09428892315983045, 0.09426131133736007, 0.09593009952248781, 0.09412299734341109, 0.09641944516698456, 0.242622855502861, 0.0939598365169349]"
215369,2010035,439612646_0096,amc,2020,3850656,439612646,ze_batch1,hinke,False,False,SNAQ,"[0, 0, 0, 0, 0, 0, 0, 1, 0]",SNAQ,"[0, 0, 0, 0, 0, 0, 0, 1, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","[0.09408812164582012, 0.09325137302044814, 0.09361922583452723, 0.09412424598685928, 0.09553081742183543, 0.09316673213830926, 0.09420988073532584, 0.24872980397586947, 0.09327979924100525]"
216714,2022704,412486717_0008,amc,2020,1644335,412486717,ze_batch1,hinke,False,False,"37,4 ° C | Ademhal ."

In [5]:
test_preds['domain_pred'] = ""

In [6]:
test_preds['domain_gold'] = ""

In [7]:
for ri, row in test_preds.iterrows():
    if row['pred_lq_data1'] == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'ADM'
    if row['pred_lq_data1'] == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'ATT'
    if row['pred_lq_data1'] == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'BER'
    if row['pred_lq_data1'] == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'ENR'
    if row['pred_lq_data1'] == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'ETN'
    if row['pred_lq_data1'] == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'FAC'
    if row['pred_lq_data1'] == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'INS'
    if row['pred_lq_data1'] == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        test_preds.loc[ri, 'domain_pred'] = 'MBW'
    if row['pred_lq_data1'] == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        test_preds.loc[ri, 'domain_pred'] = 'STM'
    #if row['pred_lq_data1'] == [0, 0, 0, 0, 0, 0, 0, 0, 0]:
     #   test_preds.loc[ri, 'domain_pred'] = 'O'

In [8]:
for ri, row in test_preds.iterrows():
    if row['original_labels'] == [1, 0, 0, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'ADM'
    if row['original_labels'] == [0, 1, 0, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'ATT'
    if row['original_labels'] == [0, 0, 1, 0, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'BER'
    if row['original_labels'] == [0, 0, 0, 1, 0, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'ENR'
    if row['original_labels'] == [0, 0, 0, 0, 1, 0, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'ETN'
    if row['original_labels'] == [0, 0, 0, 0, 0, 1, 0, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'FAC'
    if row['original_labels'] == [0, 0, 0, 0, 0, 0, 1, 0, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'INS'
    if row['original_labels'] == [0, 0, 0, 0, 0, 0, 0, 1, 0]:
        test_preds.loc[ri, 'domain_gold'] = 'MBW'
    if row['original_labels'] == [0, 0, 0, 0, 0, 0, 0, 0, 1]:
        test_preds.loc[ri, 'domain_gold'] = 'STM'
    #if row['original_labels'] == [0, 0, 0, 0, 0, 0, 0, 0, 0]:
     #   test_preds.loc[ri, 'domain_gold'] = 'O'

In [9]:
test_preds

,index,pad_sen_id,institution,year,MDN,NotitieID,batch,annotator,background_sent,target_sent,text_raw,original_labels,text,labels_9,labels_binary,labels_10,pred_best_model,pred_lq_data1,confidence,domain_pred,domain_gold
821,6810,237672660_0033,amc,2018,6223307,237672660,week_18,katsburg,False,False,"Anamnese M is wel nerveus over dit gesprek , bang dat we iets doorvertellen aan instanties .","[0, 0, 0, 0, 0, 0, 0, 0, 1]","Anamnese M is wel nerveus over dit gesprek , bang dat we iets doorvertellen aan instanties .","[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09358591870924066, 0.09325441701394238, 0.09341207319591131, 0.09336631342654787, 0.0936135816012269, 0.09385391033891609, 0.09356140714683492, 0.09322195570256535, 0.2521304228648145]",STM,STM
827,6895,237672660_0039,amc,2018,6223307,237672660,week_18,katsburg,False,False,"M is niet bang , wil ook geen aangifte of scheiding .","[0, 0, 0, 0, 0, 0, 0, 0, 1]","M is niet bang , wil ook geen aangifte of scheiding .","[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09358507025914722, 0.09324724125140077, 0.09344117951243254, 0.0933398217249897, 0.09361093802412519, 0.09388729656664592, 0.09355342069501008, 0.09321633326790207, 0.2521186986983465]",STM,STM
831,6939,237672660_0043,amc,2018,6223307,237672660,week_18,katsburg,False,False,"Heeft het vechten nooit gezien , maar ziet wel dat moeder dan boos is , heeft het mogelijk ook gehoord .","[0, 0, 0, 0, 0, 0, 0, 0, 1]","Heeft het vechten nooit gezien , maar ziet wel dat moeder dan boos is , heeft het mogelijk ook gehoord .","[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09363395734373231, 0.0932535713632954, 0.0934284485580349, 0.09333472148369376, 0.0935956190175647, 0.09388262568354304, 0.09352878612617939, 0.09321331525524418, 0.2521289551687123]",STM,STM
832,6960,237672660_0044,amc,2018,6223307,237672660,week_18,katsburg,False,False,"Priscilla is nooit geslagen , P is niet boos op de kinderen , alleen op M. M slaat ook niet .","[0, 0, 0, 0, 0, 0, 0, 0, 0]","PERSON is nooit geslagen , P is niet boos op de kinderen , alleen op PERSON slaat ook niet .","[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09361482316815592, 0.09321437383703894, 0.09343594688594525, 0.09331599384787412, 0.09360579156821942, 0.09402368259885356, 0.09352643200503734, 0.09318338190398291, 0.25207957418489246]",STM,
834,6991,237672660_0046,amc,2018,6223307,237672660,week_18,katsburg,False,False,Priscilla lijkt wel bezorgd .,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",PERSON lijkt wel bezorgd .,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.09359741542082294, 0.09330687468359604, 0.09344553446433176, 0.09336850003847856, 0.0935984449841024, 0.09377175291195286, 0.09353798381742151, 0.09326806034041701, 0.2521054333388768]",STM,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215368,2010031,439612646_0095,amc,2020,3850656,439612646,ze_batch1,hinke,False,False,Ondervoeding - SNAQ Score,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",Ondervoeding - SNAQ Score,"[0, 0, 0, 0, 0, 0, 0, 1, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","[0.09438871878799326, 0.09400581266213694, 0.09428892315983045, 0.09426131133736007, 0.09593009952248781, 0.09412299734341109, 0.09641944516698456, 0.242622855502861, 0.0939598365169349]",MBW,
215369,2010035,439612646_0096,amc,2020,3850656,439612646,ze_batch1,hinke,False,False,SNAQ,"[0, 0, 0, 0, 0, 0, 0, 1, 0]",SNAQ,"[0, 0, 0, 0, 0, 0, 0, 1, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",1,"[0, 0, 0, 0, 0, 0, 0, 1, 0]","[0.09408812164582012, 0.09325137302044814, 0.09361922583452723, 0.09412424598685928, 0.09553081742183543, 0.09316673213830926, 0.09420988073532584, 0.24872980397586947, 0.09327979924100525]",MBW,MBW
216714,2022704,412486717_000

In [10]:
y_pred = test_preds['domain_pred'].tolist()
y_true = test_preds['domain_gold'].tolist()

In [11]:
unique_labels = np.unique([y_true, y_pred])
cmtx = pd.DataFrame(
    confusion_matrix(y_true, y_pred, labels=unique_labels),
    index=['true:{:}'.format(x) for x in unique_labels],
    columns=['pred:{:}'.format(x) for x in unique_labels]
)

cmtx

,pred:,pred:ADM,pred:ATT,pred:BER,pred:ENR,pred:ETN,pred:FAC,pred:INS,pred:MBW,pred:STM
true:,100,91,2,6,17,112,98,43,30,55
true:ADM,16,164,0,0,1,0,1,0,0,0
true:ATT,0,0,8,0,0,0,1,0,0,0
true:BER,0,0,0,7,0,0,0,0,0,0
true:ENR,17,0,0,0,37,0,0,0,0,0
true:ETN,2,1,0,0,0,80,1,0,0,0
true:FAC,7,0,0,0,0,0,78,0,0,0
true:INS,8,1,0,0,0,0,4,23,0,0
true:MBW,1,0,0,0,0,0,0,0,39,0
true:STM,1,0,0,0,0,0,0,0,0,96


In [12]:
cf = cmtx.to_numpy()

In [13]:
tp = np.diag(cf)

precision = list(map(truediv, tp, np.sum(cf, axis = 0)))
recall = list(map(truediv, tp, np.sum(cf, axis = 1)))
f1 = []

for p, r in list(zip(precision, recall)):
    f = (2*p*r)/(p+r)
    f1.append(f)

#print('Precision: {}\nRecall: {}\nF1 score: {}'.format(precision, recall, f1))

In [14]:
results = pd.DataFrame([precision,
                       recall,
                       f1],
                      columns=['O', 'ADM', 'ATT', 'BER', 'ENR', 'ETN', 'FAC', 'INS', 'MBW', 'STM'],
                      index=['Precision','Recall', 'F1 score'])

results

,O,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM
Precision,0.657895,0.638132,0.800000,0.538462,0.672727,0.416667,0.426230,0.348485,0.565217,0.635762
Recall,0.180505,0.901099,0.888889,1.000000,0.685185,0.952381,0.917647,0.638889,0.975000,0.989691
F1 score,0.283286,0.747153,0.842105,0.700000,0.678899,0.579710,0.582090,0.450980,0.715596,0.774194


In [15]:
results.to_pickle("/mnt/data/Users/A-PROOF/Cecilia_S/models/fine_tuned/best_model/results_lq1.pkl")

In [16]:
domains = ['O', 'ADM', 'ATT', 'BER', 'ENR', 'ETN', 'FAC', 'INS', 'MBW', 'STM']

for d, p, r, f in list(zip(domains, precision, recall, f1)):
    print('{}\nPrecision: {}\nRecall: {}\nF1 score: {}\n'.format(d, p, r, f))

O
Precision: 0.6578947368421053
Recall: 0.18050541516245489
F1 score: 0.28328611898017

ADM
Precision: 0.6381322957198443
Recall: 0.9010989010989011
F1 score: 0.7471526195899773

ATT
Precision: 0.8
Recall: 0.8888888888888888
F1 score: 0.8421052631578948

BER
Precision: 0.5384615384615384
Recall: 1.0
F1 score: 0.7000000000000001

ENR
Precision: 0.6727272727272727
Recall: 0.6851851851851852
F1 score: 0.6788990825688074

ETN
Precision: 0.4166666666666667
Recall: 0.9523809523809523
F1 score: 0.5797101449275361

FAC
Precision: 0.4262295081967213
Recall: 0.9176470588235294
F1 score: 0.582089552238806

INS
Precision: 0.3484848484848485
Recall: 0.6388888888888888
F1 score: 0.45098039215686275

MBW
Precision: 0.5652173913043478
Recall: 0.975
F1 score: 0.7155963302752294

STM
Precision: 0.6357615894039735
Recall: 0.9896907216494846
F1 score: 0.7741935483870968

